In [168]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [169]:
#Calculate total number of players in the game according to the length of values in screenname column of dataframe
numbOfPlayers = pd.DataFrame({"Total Players":[len(purchase_data["SN"].value_counts())]})
# Display of total number of players
numbOfPlayers

,Total Players
0,576


In [170]:
#Calculate number of unique items, average price, number of purchases, and total revenue from the purchase data
itemsumdf = pd.DataFrame({"Number of Unique Items": [len(purchase_data["Item ID"].unique())], "Average Price": [round(purchase_data["Price"].mean(),2)], "Number of Purchases": [(purchase_data["Purchase ID"].count())], "Total Revenue": [(purchase_data["Price"].sum())]})
# Display of basic analysis of purchases of unique items

itemsumdf

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


In [171]:
#Drop all duplicate entries in the purchase data's screen name column according to age and gender
dropdoubledf = purchase_data[["SN", "Age", "Gender"]].drop_duplicates()

In [172]:
# Display of total players by gender
#Calculate player data according to gender value counts
genSummaryDF = pd.DataFrame({"Total Players":dropdoubledf["Gender"].value_counts(), "Percentage":dropdoubledf["Gender"].value_counts() / len(purchase_data["SN"].value_counts()) * 100})
genSummaryDF
#Format the percentage column
genSummaryDF.style.format({"Percentage":"{:.3f}%"})


,Total Players,Percentage
Male,484,84.028%
Female,81,14.062%
Other / Non-Disclosed,11,1.910%


In [173]:
dfByGen = purchase_data.groupby("Gender")
dfGenPurchases = pd.DataFrame({"Purchase Count":dfByGen["Purchase ID"].count(), 
                                    "Average Purchase Price":dfByGen["Price"].mean().map('${:,.2f}'.format),
                                    "Total Purchase Price":dfByGen["Price"].sum().map('${:,.2f}'.format),
                                    "Total Average Purchase per Person": (dfByGen["Price"].sum() / dropdoubledf["Gender"].value_counts()).map('${:,.2f}'.format)})
# Display of gender analysis of purchases and average purchase cost per person

dfGenPurchases

,Purchase Count,Average Purchase Price,Total Purchase Price,Total Average Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [174]:
#Create bins for the desired age categories and group by age demographics
purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"], [0, 9, 14, 19, 24, 29, 34, 39, 100], labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])
dfByAge = purchase_data.groupby("Age Demographics")
#Calculate percentage of players found in each age bin created
dfAge = pd.DataFrame({"Total Count":dfByAge["SN"].nunique(),
                         "Percentage of Players":dfByAge["SN"].nunique() / len(purchase_data["SN"].value_counts()) * 100})
#Format the players column with 3 decimals and percent
dfAge.style.format({"Percentage of Players":"{:.3f}%"})

,Total Count,Percentage of Players
Age Demographics,,
<10,17,2.951%
10-14,22,3.819%
15-19,107,18.576%
20-24,258,44.792%
25-29,77,13.368%
30-34,52,9.028%
35-39,31,5.382%
40+,12,2.083%


In [175]:
#Calculate purchase count, average purchase price, and total purchase price per person according to the age demographics bins
agepd = pd.DataFrame({"Purchase Count":dfByAge["Purchase ID"].count(), 
                                    "Average Purchase Price":(dfByAge["Price"].mean()).map('${:,.2f}'.format),
                                    "Total Purchase Price":(dfByAge["Price"].sum()).map('${:,.2f}'.format),
                                    "Total Average Purchase per Person":(dfByAge["Price"].sum() / dfByAge["SN"].nunique()).map('${:,.2f}'.format)})
#Display of purchasing data according the age demographic bins created
agepd

,Purchase Count,Average Purchase Price,Total Purchase Price,Total Average Purchase per Person
Age Demographics,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [176]:
TopSpendingPlayers = purchase_data.groupby("SN")
#Calculate top spending players total purchase count, average purchase price, and total purchase value
NumOneSpender = pd.DataFrame({"Purchase Count":TopSpendingPlayers["SN"].count(),
                            "Average Purchase Price":TopSpendingPlayers["Price"].mean(),
                            "Total Purchase Value":TopSpendingPlayers["Price"].sum()})
#Sort the dataframe according total purchase values descending
dfSpendSort = NumOneSpender.sort_values(["Total Purchase Value"], ascending=False).head()
#Display of top spending players with screennames sorted by total purchase value
dfSpendSort.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [177]:
updateddf = purchase_data[['Item ID', 'Item Name','Price']]
itemdf = updateddf.groupby(['Item ID', 'Item Name']).count()
#update data frame to display item purchase count instead
itemdf = itemdf.rename(columns={"Price":"Purchase Count"})

iddf = updateddf.groupby(['Item ID', 'Item Name']).sum()
iddf = iddf.rename(columns={"Price":"Total Purchase Value"})
#concatenate the two portions of item data into one dataframe for object purchase data
concatdf = pd.concat([itemdf, iddf], axis=1)
concatdf['Item Price'] = concatdf['Total Purchase Value']/concatdf['Purchase Count']
#Sort dataframe according the purchase count
mostPop = concatdf.sort_values("Purchase Count", ascending=False)
mostPop = mostPop[['Purchase Count','Item Price','Total Purchase Value']]

mostPop["Item Price"] = mostPop["Item Price"].map('${:,.2f}'.format)
mostPop["Total Purchase Value"] = mostPop["Total Purchase Value"].map('${:,.2f}'.format)
#Display of most popular purchase items sorted by purchase count totals
mostPop.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [178]:
#sort the dataframe by descending total purchase value
mostProf = concatdf.sort_values("Total Purchase Value", ascending=False)
mostProf = mostProf[['Purchase Count','Item Price','Total Purchase Value']]
#format item price into dollar format
mostProf["Item Price"] = mostProf["Item Price"].map('${:,.2f}'.format)
#format total purchase value into dollar format
mostProf["Total Purchase Value"] = mostProf["Total Purchase Value"].map('${:,.2f}'.format)
#Display of most profitables items in the game sorted by total purchase value in descending order
mostProf.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
